In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeGroupAndAgeDatasetKL, AgeGroupKLAndAgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

In [3]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation_simple.csv", "./newAugmentationDataset/")
df_train = pd.concat([df_train, aug], ignore_index=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

label_map_v = CSVUtils.get_label_map_vector()
cd_train = AgeGroupKLAndAgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                                    label_map_vector=label_map_v, transform_func=transform_func)

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=6, prefetch_factor=4)

dm_train = CustomDataLoader(cd_train)
dl_train = dm_train.get_unbalanced_dataloader(batch_size=128, shuffle=True, drop_last=True, num_workers=16, prefetch_factor=4, pin_memory=True)

In [4]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 12
####################################################

backbone = BackBone(pretrained=True, freeze=False)
backbone.train()
model_age_group = ResNetFiLMed(backbone, 8)
model_age_group.train()
model_age = ResNetFiLMed(backbone, 81)
model_age.train()
model_age_group.load_state_dict(torch.load("./model_age_group_film_feature_mae2.pt"))
model_age.load_state_dict(torch.load("./model_age_film_feature_mae2.pt"))

opt = optim.SGD(set([*model_age_group.parameters(), *model_age.parameters()]), lr=1e-4, weight_decay=1e-4)
# scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

In [5]:
best_val_aar = -1
knowledge_age_group = torch.tensor([[0.125]*8]*128, requires_grad=False).float().to("cuda")

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age_group = y[0].to("cuda")
            y_age = y[1].to("cuda")
            y_age_kl: torch.Tensor = y[2].to("cuda")
            
            knowledge = model_age_group(x, knowledge_age_group)
            loss_age_group: torch.Tensor = kl(F.log_softmax(knowledge, dim=-1), y_age_group)

            knowledge = F.softmax(knowledge, dim=-1)
            out_age = model_age(x, knowledge)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age: torch.Tensor = torch.mean(torch.abs(out - y_age))

            loss = loss_age_group + loss_age_kl + loss_age
            loss.backward()
            opt.step()
            # scheduler.step()

            tepoch.set_postfix(loss_age_group=loss_age_group.detach().cpu().numpy(), 
                                loss_age=loss_age.detach().cpu().numpy(), 
                                loss_age_kl=loss_age_kl.detach().cpu().numpy())

    def forward_function(x):
        knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
        knowledge = model_age_group(x, knowledge_age_group)
        knowledge = F.softmax(knowledge, dim=-1)
        out = model_age(x, knowledge)
        out = F.softmax(out, dim=-1)
        return out

    mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
    print(mae, val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age_group.state_dict(), "./model_age_group_film_feature.pt")
        torch.save(model_age.state_dict(), "./model_age_film_feature.pt")
        print("Saved model")

100%|██████████| 4493/4493 [02:55<00:00, 25.66 batch/s]


tensor(2.6681, dtype=torch.float64) tensor(6.4772, dtype=torch.float64) tensor(6.7083, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [02:44<00:00, 27.30 batch/s]


tensor(2.7984, dtype=torch.float64) tensor(6.3034, dtype=torch.float64) tensor(6.4783, dtype=torch.float64)


100%|██████████| 4493/4493 [02:43<00:00, 27.42 batch/s]


tensor(2.8327, dtype=torch.float64) tensor(6.3468, dtype=torch.float64) tensor(6.4871, dtype=torch.float64)


 50%|█████     | 2089/4165 [09:10<08:39,  4.00 batch/s, loss_age=2.5156956, loss_age_group=0.39369142277514557, loss_age_kl=1.0934000766383138]

In [5]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=128, num_workers=16, prefetch_factor=4, pin_memory=True)

model_age_group.load_state_dict(torch.load("./model_age_group_film_feature.pt"))
model_age_group.requires_grad_(False)
model_age.load_state_dict(torch.load("./model_age_film_feature.pt"))
opt = optim.SGD(set([*model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)

In [6]:
def forward_function(x):
    knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
    knowledge = model_age_group(x, knowledge_age_group)
    knowledge = F.softmax(knowledge, dim=-1)
    out = model_age(x, knowledge)
    out = F.softmax(out, dim=-1)
    return out

mae, val_aar, val_aar_old = validator.validate_ext(forward_function)
print(mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [02:53<00:00, 25.94 batch/s]


tensor(2.1877, dtype=torch.float64) tensor(4.1028, dtype=torch.float64) tensor(5.3027, dtype=torch.float64)


In [7]:
best_val_aar = val_aar
knowledge_age_group = torch.tensor([[0.125]*8]*128, requires_grad=False).float().to("cuda")

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age_group = y[0].to("cuda")
            y_age = y[1].to("cuda")
            y_age_kl: torch.Tensor = y[2].to("cuda")
            
            knowledge = model_age_group(x, knowledge_age_group).detach()
            #loss_age_group: torch.Tensor = kl(F.log_softmax(knowledge, dim=-1), y_age_group)

            knowledge = F.softmax(knowledge, dim=-1)
            out_age = model_age(x, knowledge)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age: torch.Tensor = torch.mean(torch.abs(out - y_age))

            loss = loss_age_kl + (torch.square(loss_age - mae) if loss_age > mae else torch.tensor(0)) # + loss_age_group + loss_age_kl
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(#loss_age_group=loss_age_group.detach().cpu().numpy(), 
                                loss_age=loss_age.detach().cpu().numpy(),
                                loss_age_kl=loss_age_kl.detach().cpu().numpy())

    def forward_function(x):
        knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
        knowledge = model_age_group(x, knowledge_age_group)
        knowledge = F.softmax(knowledge, dim=-1)
        out = model_age(x, knowledge)
        out = F.softmax(out, dim=-1)
        return out

    mae_, val_aar, val_aar_old = validator.validate_ext(forward_function)
    print(mae_, val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age_group.state_dict(), "./model_age_group_film_classifier.pt")
        torch.save(model_age.state_dict(), "./model_age_film_classifier.pt")
        print("Saved model")

100%|██████████| 4493/4493 [02:48<00:00, 26.74 batch/s]


tensor(3.6169, dtype=torch.float64) tensor(5.8213, dtype=torch.float64) tensor(5.7452, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [02:48<00:00, 26.59 batch/s]


tensor(5.0291, dtype=torch.float64) tensor(3.3203, dtype=torch.float64) tensor(2.8767, dtype=torch.float64)


100%|██████████| 4493/4493 [02:49<00:00, 26.47 batch/s]


...

tensor(3.7507, dtype=torch.float64) tensor(5.8628, dtype=torch.float64) tensor(5.6012, dtype=torch.float64)


100%|██████████| 4493/4493 [02:51<00:00, 26.25 batch/s]


tensor(3.5680, dtype=torch.float64) tensor(6.1501, dtype=torch.float64) tensor(5.9457, dtype=torch.float64)


  2%|▏         | 94/4165 [00:40<29:20,  2.31 batch/s, loss_age=3.267005, loss_age_kl=1.3268219145251972]   


KeyboardInterrupt: 

In [ ]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=128, num_workers=16, prefetch_factor=4, pin_memory=True)

model_age_group.load_state_dict(torch.load("./model_age_group_film_classifier_6.pt"))
model_age_group.requires_grad_(False)
model_age.load_state_dict(torch.load("./model_age_film_classifier_6.pt"))
opt = optim.SGD(set([*model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)

In [ ]:
def forward_function(x):
    knowledge_age_group = torch.tensor([[0.125]*8]*len(x), requires_grad=False).float().to("cuda")
    knowledge = model_age_group(x, knowledge_age_group)
    knowledge = F.softmax(knowledge, dim=-1)
    out = model_age(x, knowledge)
    out = F.softmax(out, dim=-1)
    return out

ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
print(ae, mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [02:43<00:00, 27.44 batch/s]


{0: tensor(3.4816, dtype=torch.float64), 1: tensor(4.1946, dtype=torch.float64), 2: tensor(3.5825, dtype=torch.float64), 3: tensor(3.7497, dtype=torch.float64), 4: tensor(3.9872, dtype=torch.float64), 5: tensor(3.1775, dtype=torch.float64), 6: tensor(2.8100, dtype=torch.float64), 7: tensor(3.2310, dtype=torch.float64)} tensor(3.6658, dtype=torch.float64) tensor(6.0273, dtype=torch.float64) tensor(5.8883, dtype=torch.float64)
